In [11]:
### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
import pandas as pd


In [2]:
# Create the engine to connect to a sqlite file.
engine = create_engine("sqlite:///db/CCES_Ver61.sqlite")

# Declare a base using automap_base()
Base = automap_base()


Base.prepare(engine, reflect=True)

# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

['Cces_16']

In [3]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.Cces_16

In [4]:
# Create a variable to manage session.queries later on...
session = Session(engine)
# session = Session(bind = engine)

# pd.query testing

In [9]:
stmt = session.query(CCES).statement
print(stmt)

SELECT "Cces_16".index, "Cces_16"."V101", "Cces_16".lookupzip, "Cces_16"."StateName", "Cces_16"."StateAbb", "Cces_16"."Latitude", "Cces_16"."Longitude", "Cces_16".countyfips, "Cces_16".birthyr, "Cces_16".gender, "Cces_16".educ, "Cces_16"."GunBackgroundChecks_16", "Cces_16"."ProhibitPublication_16", "Cces_16"."BanAssultWeapons_16", "Cces_16"."MakeCCPEasier_16", "Cces_16"."AlwaysAllowChoice_16", "Cces_16"."RapeIncestorHealth_16", "Cces_16"."ProhibitMoreThan20Weeks_16", "Cces_16"."Employersdeclinebenefits_16", "Cces_16"."ProhibitFedFunds_16", "Cces_16"."GayMarriage" 
FROM "Cces_16"


In [14]:
df = pd.read_sql_query(stmt, session.bind)
df.head(2)

,index,V101,lookupzip,StateName,StateAbb,Latitude,Longitude,countyfips,birthyr,gender,...,GunBackgroundChecks_16,ProhibitPublication_16,BanAssultWeapons_16,MakeCCPEasier_16,AlwaysAllowChoice_16,RapeIncestorHealth_16,ProhibitMoreThan20Weeks_16,Employersdeclinebenefits_16,ProhibitFedFunds_16,GayMarriage
0,0,222168628,3442,New Hampshire,NH,44.001231,-71.579923,33011.0,1969,female,...,Support,Oppose,Oppose,Oppose,Support,Oppose,Support,Oppose,Oppose,Support
1,1,301400441,3303,New Hampshire,NH,44.001231,-71.579923,33013.0,1948,female,...,Support,Oppose,Support,Oppose,Support,Oppose,Oppose,Oppose,Oppose,Support


In [15]:
print(list(df.columns)[11:])

['GunBackgroundChecks_16', 'ProhibitPublication_16', 'BanAssultWeapons_16', 'MakeCCPEasier_16', 'AlwaysAllowChoice_16', 'RapeIncestorHealth_16', 'ProhibitMoreThan20Weeks_16', 'Employersdeclinebenefits_16', 'ProhibitFedFunds_16', 'GayMarriage']


In [21]:
type(stmt)



sqlalchemy.sql.annotation.AnnotatedSelect

# List of dropdown variables

In [ ]:
#############################################
# Dropdown Variables mapped to Sql...
#############################################

# Gun Questions
GunBackgroundChecks_16 = CCES.GunBackgroundChecks_16
ProhibitPublication_16 = CCES.ProhibitPublication_16
BanAssultWeapons_16 = CCES.BanAssultWeapons_16
MakeCCPEasier_16 = CCES.MakeCCPEasier_16

# Abortion Questions
AlwaysAllowChoice_16 = CCES.AlwaysAllowChoice_16
RapeIncestorHealth_16 = CCES.RapeIncestorHealth_16
ProhibitMoreThan20Weeks_16 = CCES.ProhibitMoreThan20Weeks_16
Employersdeclinebenefits_16 = CCES.Employersdeclinebenefits_16
ProhibitFedFunds_16 = CCES.ProhibitFedFunds_16

# Gay Marriage Question
GayMarriage = CCES.GayMarriage
#############################################

# Flask Route - metadata totals testing

In [ ]:
#############################################
# Creating metadata totals function for metadata totals flask route
#############################################

In [ ]:
#############################################
# Function to return Totals - Total Support and Total Oppose for each issue
#############################################
def sample_metadatatotals(sample):
    """Return the totals for a given sample"""
    TotalResults = []
    
    Approve = {}
    Oppose = {}
    
    ApproveTemp = session.query(sample).filter(sample == 'Support').count()
    Approve['Approve'] = ApproveTemp
    TotalResults.append(Approve)
    
    OpposeTemp = session.query(sample).filter(sample == 'Oppose').count()
    Oppose['Oppose'] = OpposeTemp
    TotalResults.append(Oppose)

    print(TotalResults)
#     return jsonify(TotalResults)

In [ ]:
#############################################
# Metadatatotals Function Testing 1

sample_metadatatotals(GayMarriage)

In [ ]:
#############################################
# Metadatatotals Function Testing 2

sample_metadatatotals(CCES.GunBackgroundChecks_16)

In [ ]:
#############################################
# Metadatatotals Function - without function - direct query to verify results

ApproveGay = session.query(CCES.GayMarriage).filter(CCES.GayMarriage == 'Support').count()
OpposeGay = session.query(CCES).filter(CCES.GayMarriage == 'Oppose').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Sqlite query of votes opposint Gay Marriage: {OpposeGay}')

# Flask Route - Metadata States Groupby Results

In [ ]:
#############################################
# Metadata States Groupby Function
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    StatesFor = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Support').\
                group_by(CCES.StateName).all()
    StatesNot = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Oppose').\
                group_by(CCES.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = []
    
    for i, j in zip(StatesFor, StatesNot):

        tempfile = {}
        tempfile['State'] = i[0]
        tempfile['Support'] = i[1]
        tempfile['Oppose'] = j[1]
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        tempfile['Overall'] = tempoverall

        States_Results.append(tempfile)

    return States_Results
    
            

In [ ]:
#############################################
# Metadata States Function Testing 1

Metadata_States(GunBackgroundChecks_16)

In [ ]:
#############################################
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Support').group_by(CCES.StateName).all()
StatesNot = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Oppose').group_by(CCES.StateName).all()
print(StatesFor[0:5])
print(StatesNot[0:5])

In [ ]:
#############################################
# Groupby Function testing before building function
#############################################
StatesForAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Support').group_by(CCES.StateName).all()
StatesNotAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Oppose').group_by(CCES.StateName).all()

ThirdTry = []

for i, j in zip(StatesForAbortion, StatesNotAbortion):

    tempfile = {}
    tempfile['State'] = i[0]
    tempfile['Support'] = i[1]
    tempfile['Oppose'] = j[1]
    if i[1] > j[1]:
        tempoverall = 'Support'
    else:
        tempoverall = 'Oppose'
    tempfile['Overall'] = tempoverall
        
    ThirdTry.append(tempfile)
ThirdTry

In [ ]:
#############################################
# Testing Groupby by specific state
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################
AlabamaTotalExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').count()
AlabamaForExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Support').count()
AlabamaAgainstExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Oppose').count()
print(AlabamaTotalExample)
print(AlabamaForExample)
print(AlabamaAgainstExample)